# self_defined_softmax

# import

In [1]:

import torch
print(torch.__version__)

1.1.0


In [2]:
import os
import random
from tqdm import tqdm
from PIL import Image, ImageFilter
from torchvision.transforms import *
import math
import numpy as np
import torch
import os
import random
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter, ImageOps
import cv2
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

# import visdom

import argparse, time

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
from torchvision import transforms, models
import torch.backends.cudnn as cudnn
from torch.nn.parallel.data_parallel import data_parallel
import torchvision
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
# import visdom
# import pretrainedmodels

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn

from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torch.nn.parallel.data_parallel import data_parallel
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable

In [3]:
ls ../input


designed_image_test_v2/  designer_image_train_v2_cropped/  sample.csv


# config

In [4]:
class DefaultConfigs(object):
    data_root = '../input'
    train_root = '../input/designer_image_train_v2_cropped/designer_image_train_v2_cropped' 
    test_root = '../input/designed_image_test_v2/test'
    
    model = 'ResNet18' # ResNet152 使用的模型
    freeze = True # 是否冻结卷基层

    seed = 1000 # 固定随机种子
    num_workers = 2 # DataLoader 中的多线程数量
    num_classes = 50 # 分类类别数
    num_epochs = 60
    batch_size = 64 # 128 48
    lr = 0.01 # 初始lr
    width = 256 # 输入图像的宽
    height = 256 # 输入图像的高
    iter_smooth = 1 # 打印&记录log的频率

    # OHEM
    OHEM = False
    OHEM_ratio = 0.1

    # resume checkpoint
    resume = False #
    checkpoint = 'ResNet18.pth' # 训练完成的模型名
    # checkpoint = 'ResNet18.pth' # 训练完成的模型名
    # smooth label
    smooth_label = False

config = DefaultConfigs()

# creat img list

In [5]:
random.seed(config.seed)

if not os.path.exists('./data'):
    os.makedirs('./data')

train_txt = open('./data/train.txt', 'w')
val_txt = open('./data/valid.txt', 'w')
test_txt = open('./data/test.txt', 'w')
label_txt = open('./data/label_list.txt', 'w')

# creat label list
label_list = []
for dir in os.listdir(config.train_root):
    if dir not in label_list:
        label_list.append(dir)
        label_txt.write('{}  {}\n'.format(dir,len(label_list)))

def creat_train_txt(root = config.train_root):
    i = 0
    for i in range(len(label_list)):
        im_path = os.path.join(root,label_list[i])
        train_list = random.sample(os.listdir(im_path), int(len(os.listdir(im_path))*0.9))
        for im in train_list:
            train_txt.write('{}/{}  {}\n'.format(label_list[i], im, i))
        for im in os.listdir(im_path):
            if im not in train_list:
                val_txt.write('{}/{}  {}\n'.format(label_list[i], im, i))
        i += 1
            
def creat_test_txt(root = config.test_root):
    test_list = os.listdir(root)
    print(len(test_list))
    for im in test_list:
        test_txt.write('{}  {}\n'.format(im, 0))
        
creat_train_txt()
creat_test_txt()



5325


In [6]:
random.seed(config.seed)

if not os.path.exists('./data'):
    os.makedirs('./data')

train_txt = open('./data/train.txt', 'w')
val_txt = open('./data/valid.txt', 'w')
test_txt = open('./data/test.txt', 'w')
label_txt = open('./data/label_list.txt', 'w')

# creat label list
label_list = []
for dir in os.listdir(config.train_root):
    if dir not in label_list:
        label_list.append(dir)
        label_txt.write('{}  {}\n'.format(dir,len(label_list)))

def creat_train_txt(root = config.train_root):
    i = 0
    for i in range(len(label_list)):
        im_path = os.path.join(root,label_list[i])
        train_list = random.sample(os.listdir(im_path), int(len(os.listdir(im_path))*0.9))
        for im in train_list:
            train_txt.write('{}/{}  {}\n'.format(label_list[i], im, i))
        for im in os.listdir(im_path):
            if im not in train_list:
                val_txt.write('{}/{}  {}\n'.format(label_list[i], im, i))
        i += 1
            
def creat_test_txt(root = config.test_root):
    test_list = os.listdir(root)
    print(len(test_list))
    for im in test_list:
        test_txt.write('{}  {}\n'.format(im, 0))
        
creat_train_txt()
creat_test_txt()



5325


# see txt

In [7]:
with open('./data/train.txt') as f1:
    lines1 = f1.readlines()
    print('train:',len(lines1))

with open('./data/valid.txt') as f2:
    lines2 = f2.readlines()
    print('valid:',len(lines2))

with open('./data/test.txt') as f3:
    lines3 = f3.readlines()
    print('test:',len(lines3))
    
    
with open('./data/label_list.txt') as f4:
    lines4 = f4.readlines()
    print('label list:',len(lines4))
    

   

train: 40574
valid: 4533
test: 5325
label list: 50


# augmentation

In [8]:
def RandomErasing(im, probability = 0.5, sl = 0.02, sh = 0.4, r1 = 0.3, mean=[128, 128, 128]):
    '''
    performs Random Erasing in Random Erasing Data Augmentation by Zhong et al. 
    -------------------------------------------------------------------------------------
    img: PIL img
    probability: The probability that the operation will be performed.
    sl: min erasing area
    sh: max erasing area
    r1: min aspect ratio
    mean: erasing value
    -------------------------------------------------------------------------------------
    '''
    if random.uniform(0, 1) > probability:
        return im

    else:
        img = np.array(im)
        area = img.shape[0] * img.shape[1]
       
        while True:
            target_area = random.uniform(sl, sh) * area
            aspect_ratio = random.uniform(r1, 1/r1)
            h = int(round(math.sqrt(target_area * aspect_ratio)))
            w = int(round(math.sqrt(target_area / aspect_ratio)))
            if img.shape[0] > w and img.shape[1] > h:
                break

        if w < img.shape[0] and h < img.shape[1]:
            x1 = random.randint(0, img.shape[0] - w)
            y1 = random.randint(0, img.shape[1] - h)
            # if img.size()[0] == 3:
            if im.mode == 'RGB':
                img[x1:x1+h, y1:y1+w, 0] = mean[0]
                img[x1:x1+h, y1:y1+w, 1] = mean[1]
                img[x1:x1+h, y1:y1+w, 2] = mean[2]
            elif im.mode == 'L':
                img[x1:x1+h, y1:y1+w] = mean[0]
        img = Image.fromarray(np.uint8(img))

        return img


class MyGaussianBlur(ImageFilter.Filter):
    name = "GaussianBlur"

    def __init__(self, radius=2):
        self.radius = radius

    def filter(self, image):
        return image.gaussian_blur(self.radius)

# dataset

In [9]:
def read_txt(path):
    ims, labels = [], []
    with open(path, 'r') as f:
        for line in f.readlines():
            im, label = line.strip().split('  ')
            ims.append(im)
            labels.append(int(label))
    return ims, labels

class iDesignerDataset(Dataset):
    def __init__(self, txt_path, transform=None):
        self.ims, self.labels = read_txt(txt_path)
        self.transform = transform

    def __getitem__(self, index):
        im_path = self.ims[index]
        label = self.labels[index]
        im_path = os.path.join(config.train_root, im_path)
        im = Image.open(im_path)
        #im = im.resize((self.width, self.height))
        if self.transform is not None:
            im = RandomErasing(im, probability=0.5, sl=0.1, sh=0.4, r1=0.3, mean=[128, 128, 128])
            if random.random() < 0.5:
                im = im.filter(MyGaussianBlur(radius=5))
            im = self.transform(im)

        return im, label

    def __len__(self):
        return len(self.ims)

def padding_image(im):
    w, h = im.size
    pad_im = Image.new('RGB', (w, h), (128, 128, 128))
    pad_im.paste(im, (0, 0))

    return pad_im

class iDesignerTestDataset(Dataset):
    def __init__(self, csv_path, transform=None, augment=None):
        df = pd.read_csv(csv_path)
        self.ims = df['Id']
        self.transform = transform
        self.augment = augment

    def __getitem__(self, index):
        name = self.ims[index]
        # im_path = os.path.join(config.data_root, 'test', name)
        im_path = os.path.join(config.data_root, 'designed_image_test_v2/test', name)
        # print(im_path)
        im = Image.open(im_path)
        im = padding_image(im)
        #im = im.resize((self.width, self.height))
        if self.augment == 2:
            im = im
        # top-left crop
        if self.augment == 1:
            w, h = im.size
            im = im.crop((0, 0, w-10, h-10))
        # center crop
        if self.augment == 0:
            w, h = im.size
            im = im.crop((10, 10, w-10, h-10))

        if self.transform is not None:
            im = self.transform(im)

        return im, name
    
    def __len__(self):
        return len(self.ims)

# self-defined softmax

In [10]:
# def self_defined_softmax(outputs):
#     # outputs = Variable(outputs,requires_grad=True)
#     batch_size = outputs.size(0)
#     num_classes = outputs.size(1)
    
#     # step 1 : exp
#     for i in range(batch_size):
#         outputs[i] = torch.exp(outputs[i])

#     # step 2 : normalize
#     for i in range(batch_size):
#         sum = torch.ones(batch_size)
#         sum[i] = torch.FloatTensor([0])
#         for j in range(num_classes):
#             sum[i] = torch.add(sum[i], outputs[i][j]).cuda()
#         outputs[i] = torch.div(outputs[i], sum[i])
    
#     return outputs

def self_defined_softmax(outputs):
    batch_size = outputs.size(0)
    for i in range(batch_size):
        outputs[i]=torch.exp(outputs[i])/torch.sum(torch.exp(outputs[i]))
    return  outputs

# loss

In [11]:
class FocalLoss(nn.Module):
    r"""
        This criterion is a implemenation of Focal Loss, which is proposed in 
        Focal Loss for Dense Object Detection.

            Loss(x, class) = - \alpha (1-softmax(x)[class])^gamma \log(softmax(x)[class])

        The losses are averaged across observations for each minibatch.

        Args:
            alpha(1D Tensor, Variable) : the scalar factor for this criterion
            gamma(float, double) : gamma > 0; reduces the relative loss for well-classiﬁed examples (p > .5), 
                                   putting more focus on hard, misclassiﬁed examples
            size_average(bool): By default, the losses are averaged over observations for each minibatch.
                                However, if the field size_average is set to False, the losses are
                                instead summed for each minibatch.


    """
    def __init__(self, class_num, alpha=None, gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        if alpha is None:
            self.alpha = Variable(torch.ones(class_num, 1))
        else:
            if isinstance(alpha, Variable):
                self.alpha = alpha
            else:
                self.alpha = Variable(alpha).cuda()
        self.gamma = gamma
        self.class_num = class_num
        self.size_average = size_average

    # inputs is the socres of output, and has 2 dims,one for batchsize, the other for num_classes
    # targets is groundtruth labels
    def forward(self, inputs, targets):
        
        N = inputs.size(0)    # number of batch size
        C = inputs.size(1)    # number of classes
        # P = F.softmax(inputs)
        P = self_defined_softmax(inputs)
        # print('after softmax: ',P)
        
        class_mask = inputs.data.new(N, C).fill_(0)
        class_mask = Variable(class_mask)
        ids = targets.view(-1, 1)
        class_mask.scatter_(1, ids.data, 1.)

        if inputs.is_cuda and not self.alpha.is_cuda:
            self.alpha = self.alpha.cuda()
        alpha = self.alpha[ids.data.view(-1)]

        probs = (P*class_mask).sum(1).view(-1,1)

        log_p = probs.log()

        batch_loss = -alpha*(torch.pow((1-probs), self.gamma))*log_p 

        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()
        return loss

class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        if torch.cuda.is_available():
            self.criterion = nn.KLDivLoss(size_average=False).cuda()
        else:
            self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size

        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist

        if torch.cuda.is_available():
            loss = self.criterion(x.cuda(), Variable(true_dist, requires_grad=False).cuda())
        else:
            loss = self.criterion(x, Variable(true_dist, requires_grad=False))

        return loss


# metric

In [12]:
def accuracy(output, target, topk=(1, 5)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

# network

In [13]:


def whitening(im):
    batch_size, channel, h, w = im.shape
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    im = torch.cat([(im[:,[0]]-mean[0])/std[0],
                    (im[:,[1]]-mean[1])/std[1],
                    (im[:,[2]]-mean[2])/std[2]], 1)
    return im

def l2_norm(x):
    norm = torch.norm(x, p=2, dim=1, keepdim=True)
    x = torch.div(x, norm)
    return x

class ResNet18(nn.Module):
    def __init__(self, model, num_classes=1000):
        super(ResNet18, self).__init__()
        self.backbone = model

        # 3 3*3 convs replace 1 7*7
        self.conv1 = nn.Conv2d(3, 16, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(16, 32, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 64, 3, stride=2, padding=1)

        # conv
        # conv replace FC
        self.conv_final = nn.Conv2d(512, num_classes, 1, stride=1)
        self.ada_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # FC
        self.fc1 = nn.Linear(512, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        # x = whitening(x)
        '''
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)

        x = self.conv3(x)
        '''
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        # FC
        x = self.backbone.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = l2_norm(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = l2_norm(x)

        '''
        # fully conv
        x = self.conv_final(x)
        x = self.ada_avg_pool(x)
        x = x.view(x.size(0), -1)
        # print x.size()
        '''


        return x

# lr

In [14]:
def half_lr(init_lr, ep):
    lr = init_lr / 2**ep

    return lr

def step_lr(ep):
    if ep < 20:
        lr = 0.005
    elif ep < 40:
        lr = 0.001
    elif epo < 50:
        lr = 0.0001
    else:
        lr = 0.00001
    return lr


def warmup_lr(init_lr, warmup_epoch, epoch):
    if epoch < warmup_epoch:
        lr = init_lr / warmup_epoch * (epoch+1)
    elif epoch < warmup_epcoh + 80:
        lr = init_lr
    elif epoch < warmup_epoch + 120:
        lr = init_lr / 10
    elif epoch < warmup_epoch + 150:
        lr = init_lr / 100
    elif epoch < warmup_epoch + 200:
        lr = init_lr / 1000
    else:
        lr = 1e-5

    return lr


# train

In [15]:
def create_weights(dataset, label_list):
    label2name = {}
    with open(label_list, 'r') as f:
        lines = f.readlines()
    for line in lines:
        name, label = line.strip().split(',')
        label2name[int(label)] = name
    name2num = {}
    path = os.path.join(config.data_root, 'designer_image_train_v2_cropped', 'designer_image_train_v2_cropped')
    for name in os.listdir(path):
        name2num[name] = len(os.listdir(os.path.join(path, name)))

    weights = []
    for im, label in dataset:
        name = label2name[label]
        num = name2num[name]
        weights.append(1./num)

    return weights
    

def train():
    # model
    if config.model == 'ResNet18_plus_plus':
        backbone = models.resnet18(pretrained=True)
        model = ResNet18_plus_plus(backbone, num_classes=config.num_classes)
    
    if config.model == 'ResNet18':
        backbone = models.resnet18(pretrained=True)
        model = ResNet18(backbone, num_classes=config.num_classes)
    elif config.model == 'ResNet34':
        backbone = models.resnet34(pretrained=True)
        model = ResNet34(backbone, num_classes=config.num_classes)
    elif config.model == 'ResNet50':
        backbone = models.resnet50(pretrained=True)
        model = ResNet50(backbone, num_classes=config.num_classes)
    elif config.model == 'ResNet101':
        backbone = models.resnet101(pretrained=True)
        model = ResNet101(backbone, num_classes=config.num_classes)
    elif config.model == 'ResNet152':
        backbone = models.resnet152(pretrained=True)
        model = ResNet152(backbone, num_classes=config.num_classes)
    elif config.model == 'se_resnet50':
        backbone = pretrainedmodels.__dict__['se_resnet50'](pretrained='imagenet')
        model = se_resnet50(backbone, num_classes=config.num_classes)
    else:
        print('ERROR: No model {}!!!'.format(config.model))
    print(model)
    # model = torch.nn.DataParallel(model)
    model.cuda()
    
    # freeze layers
    if config.freeze:
        for p in model.backbone.layer1.parameters(): p.requires_grad = False
        for p in model.backbone.layer2.parameters(): p.requires_grad = False
        for p in model.backbone.layer3.parameters(): p.requires_grad = False
        #for p in model.backbone.layer4.parameters(): p.requires_grad = False


    # loss
    # criterion = nn.CrossEntropyLoss().cuda()
    criterion = FocalLoss(config.num_classes, alpha=None, gamma=2, size_average=True)
    # criterion = LabelSmoothing(config.num_classes, 0, 0.1)

    # train data
    transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                    transforms.ColorJitter(0.05, 0.05, 0.05),
                                    transforms.RandomRotation(10),
                                    transforms.Resize((config.width, config.height)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225])])
    dst_train = iDesignerDataset('./data/train.txt', transform=transform)
#     weights = create_weights(dst_train, './data/label_list.txt')
#     sampler = WeightedRandomSampler(weights, num_samples=len(dst_train), replacement=True)
    # 会根据每个样本的权重选取数据，在样本比例不均衡的问题中，可用它来进行重采样。
#     dataloader_train = DataLoader(dst_train, shuffle=False, batch_size=config.batch_size, 
#                                   num_workers=config.num_workers, sampler=sampler)

    dataloader_train = DataLoader(dst_train, shuffle=True, batch_size=int(config.batch_size), 
                                  num_workers=config.num_workers)

    # validation data
    transform = transforms.Compose([transforms.Resize((config.width, config.height)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225])])
    dst_valid = iDesignerDataset('./data/valid.txt', transform=transform)

    dataloader_valid = DataLoader(dst_valid, shuffle=False, batch_size=int(config.batch_size/2), num_workers=config.num_workers)

    # log
    if not os.path.exists('./log'):
        os.makedirs('./log')
    log = open('./log/log.txt', 'a')

    log.write('-'*30+'\n')
    log.write('model:{}\nnum_classes:{}\nnum_epoch:{}\nim_width:{}\nim_height:{}\niter_smooth:{}\n'.format(
               config.model, config.num_classes, config.num_epochs,
               config.width, config.height, config.iter_smooth))

    # load checkpoint
    if config.resume:
        print('resume checkpoint...')
        model = torch.load(os.path.join('./checkpoints', config.checkpoint))

    # visdom
    # vis = visdom.Visdom(env='kaggle_idesigner')

    # optimizer
    # optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=0.0002)
    # optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
    #                              lr=0.00001, betas=(0.9, 0.999), weight_decay=0.0002)
    # optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
    #                       lr=lr, momentum=1e-1, weight_decay=1e-4)

    # adjust lr
    # lr = half_lr(config.lr, epoch)
    # lr = step_lr(epoch)
    # lr_scheduler = torch.optim.lr_scheduler.
    # lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
    # lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [40, 100, 150, 200], gamma=0.1)

    cudnn.benchmark = True

    # train
    sum = 0
    train_loss_sum = 0
    train_top1_sum = 0
    max_val_top1_acc = 0
    iters = 0
    for epoch in range(config.num_epochs):
        val_loss, val_top1  = eval(model, dataloader_valid, criterion)
        ep_start = time.time()
        lr = step_lr(epoch)
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                                     lr=lr, betas=(0.9, 0.999), weight_decay=0.0002)
        model.train()
        for i, (ims, label) in enumerate(dataloader_train):
            input = Variable(ims).cuda()
            target = Variable(label).cuda().long()

            output = model(input)

            if config.smooth_label:
                smoothed_target = label_smoothing(output, target).cuda()
                loss = F.kl_div(output, smoothed_target).cuda()
            
            # OHEM: online hard example mining
            if not config.OHEM and not config.smooth_label:
                loss = criterion(output, target)
            elif config.OHEM:
                if epoch < 50:
                    loss = criterion(output, target)
                else:
                    loss = F.cross_entropy(output, target, reduce=False).cuda()
                    OHEM, _ = loss.topk(int(config.num_classes*config.OHEM_ratio), dim=0, 
                                        largest=True, sorted=True)
                    loss = OHEM.mean()

            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #lr_scheduler.step()

            acc = accuracy(output.data, target.data, topk=(1,))
            train_loss_sum += loss.data.cpu().numpy()
            train_top1_sum += acc[0]
            sum += 1


            if (i+1) % config.iter_smooth == 0:
                iters += 1
#                 vis.line(X=torch.FloatTensor([iters]), Y=torch.FloatTensor([train_loss_sum/sum]),
#                          win='train_loss', update='append')
#                 vis.line(X=torch.FloatTensor([iters]), Y=torch.FloatTensor([train_top1_sum/sum]),
#                          win='train_acc_top1', update='append')

                print('Epoch [%d/%d], Iter [%d/%d], LR: %.6f, Loss: %.4f, top1: %.4f'
                       %(epoch+1, config.num_epochs, i+1, len(dst_train)//config.batch_size, lr,
                        train_loss_sum/sum, train_top1_sum/sum))
                log.write('Epoch [%d/%d], Iter [%d/%d], Loss: %.4f, top1: %.4f\n'
                           %(epoch+1, config.num_epochs, i+1, len(dst_train)//config.batch_size, 
                            train_loss_sum/sum, train_top1_sum/sum))
                sum = 0
                train_loss_sum = 0
                train_top1_sum = 0
 
        epoch_time = (time.time() - ep_start) / 60.
        if epoch % 1 == 0 and epoch < config.num_epochs:
            # eval
            val_time_start = time.time()
            val_loss, val_top1  = eval(model, dataloader_valid, criterion)
            val_time = (time.time() - val_time_start) / 60.

#             vis.line(X=torch.FloatTensor([epoch]), Y=torch.FloatTensor([val_loss]),
#                      win='val_loss', update='append')
#             vis.line(X=torch.FloatTensor([epoch]), Y=torch.FloatTensor([val_top1]),
#                      win='val_acc_top1', update='append')

            print('Epoch [%d/%d], Val_Loss: %.4f, Val_top1: %.4f, best_top1: %.4f'
                   %(epoch+1, config.num_epochs, val_loss, val_top1, max_val_top1_acc))
            print('epoch time: {} min'.format(epoch_time))
            if val_top1[0].data > max_val_top1_acc:
                max_val_top1_acc = val_top1[0].data
                print('Taking top1 snapshot...')
                if not os.path.exists('./checkpoints'):
                    os.makedirs('./checkpoints')
                torch.save(model, '{}/{}.pth'.format('checkpoints', config.model))

            log.write('Epoch [%d/%d], Val_Loss: %.4f, Val_top1: %.4f, best_top1: %.4f\n'
                       %(epoch+1, config.num_epochs, val_loss, val_top1, max_val_top1_acc))
        torch.save(model, '{}/{}_last.pth'.format('checkpoints', config.model))

    log.write('-'*30+'\n')
    log.close()

# validation
def eval(model, dataloader_valid, criterion):
    sum = 0
    val_loss_sum = 0
    val_top1_sum = 0
    model.eval().cuda()
    for ims, label in dataloader_valid:
        input_val = Variable(ims).cuda()
        target_val = Variable(label).cuda()
        output_val = model(input_val)
        loss = criterion(output_val, target_val)
        acc_val = accuracy(output_val.data, target_val.data, topk=(1,))
        
        sum += 1
        val_loss_sum += loss.data.cpu().numpy()
        val_top1_sum += acc_val[0]
    avg_loss = val_loss_sum / sum
    avg_top1 = val_top1_sum / sum
    return avg_loss, avg_top1

if __name__ == '__main__':
    '''
    transform = transforms.Compose([transforms.Resize((config.width, config.height)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225])])
    dst_train = iDesignerDataset('./data/train.txt', transform=transform)
    weights = create_weights(dst_train, './data/label_list.txt')
    print weights, len(weights)
    '''
    train()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /tmp/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:00<00:00, 133403193.28it/s]


ResNet18(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_sta

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type ResNet18. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch [2/60], Iter [1/633], LR: 0.005000, Loss: 3.6711, top1: 9.3750
Epoch [2/60], Iter [2/633], LR: 0.005000, Loss: 3.6859, top1: 6.2500
Epoch [2/60], Iter [3/633], LR: 0.005000, Loss: 3.6757, top1: 7.8125
Epoch [2/60], Iter [4/633], LR: 0.005000, Loss: 3.6838, top1: 6.2500
Epoch [2/60], Iter [5/633], LR: 0.005000, Loss: 3.7131, top1: 4.6875
Epoch [2/60], Iter [6/633], LR: 0.005000, Loss: 3.6393, top1: 10.9375
Epoch [2/60], Iter [7/633], LR: 0.005000, Loss: 3.7127, top1: 4.6875
Epoch [2/60], Iter [8/633], LR: 0.005000, Loss: 3.6767, top1: 4.6875
Epoch [2/60], Iter [9/633], LR: 0.005000, Loss: 3.6519, top1: 12.5000
Epoch [2/60], Iter [10/633], LR: 0.005000, Loss: 3.6562, top1: 7.8125
Epoch [2/60], Iter [11/633], LR: 0.005000, Loss: 3.6803, top1: 3.1250
Epoch [2/60], Iter [12/633], LR: 0.005000, Loss: 3.6851, top1: 6.2500
Epoch [2/60], Iter [13/633], LR: 0.005000, Loss: 3.6637, top1: 0.0000
Epoch [2/60], Iter [14/633], LR: 0.005000, Loss: 3.6558, top1: 6.2500
Epoch [2/60], Iter [15/633]

# inference

In [16]:
ls checkpoints

ResNet18.pth  ResNet18_last.pth


In [17]:
def read_label(path):
    data = open(path, 'r')
    label2name = {}
    for line in data.readlines():
        name, label = line.strip().split('  ')
        label2name[int(label)] = name
    return label2name


def inference():
    # model
    # load checkpoint
    model = torch.load(os.path.join('./checkpoints', config.checkpoint))
    # print model
    # model = torch.nn.DataParallel(model)
    model.cuda()
    
    # validation data
    transform = transforms.Compose([transforms.Resize((config.width, config.height)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225])])
    # label2name
    label2name = read_label('./data/label_list.txt')
    
    # TTA raw invert CenterCrop
    augments = [0, 1, 2]
    tta0, tta1, tta2 = {}, {}, {}
    for idx in range(len(augments)):
        print('TTA {}'.format(idx))
        dst_test = iDesignerTestDataset(os.path.join(config.data_root, 'sample.csv'), 
                                 transform=transform, augment=augments[idx])
        dataloader_test = DataLoader(dst_test, shuffle=False, batch_size=int(config.batch_size/2), 
                                      num_workers=config.num_workers)

        sum = 0
        model.eval()
        results = []
        probs_all = []
        for ims, im_names in tqdm(dataloader_test):
            input = Variable(ims).cuda()
            output = model(input)

            _, preds = output.topk(1, 1, True, True)
            preds = preds.cpu().detach().numpy()
            for pred, im_name in zip(preds, im_names):
                top5_name = [label2name[p] for p in pred]
                results.append({'Id':im_name, 'Category':''.join(top5_name)})

            # TTA
            probs = F.softmax(output)
            probs = probs.cpu().detach().numpy()
            for prob, im_name in zip(probs, im_names):
                if idx == 0:
                    tta0[im_name] = prob
                elif idx == 1:
                    tta1[im_name] = prob
                elif idx == 2:
                    tta2[im_name] = prob
                else:
                    print('Error: No  other TTA method!!!')
                    break
        # save no TTA
        df = pd.DataFrame(results, columns=['Id', 'Category'])
        df.to_csv('./data/result_no_TTA.csv', index=False)
    # endemble TTA
    #print tta0
    tta_results = []
    for key in tta0.keys():
        prob = (tta0[key] + tta1[key] + tta2[key]) / 3.
        top1_idx = prob.argsort()[-1]
        top1_name = label2name[top1_idx]
        tta_results.append({'Id':key, 'Category':top1_name})
    # save TTA result
    tta_df = pd.DataFrame(tta_results, columns=['Id', 'Category'])
    tta_df.to_csv('./data/result_3TTA.csv', index=False)

if __name__ == '__main__':
    inference()
    print('finished')


TTA 0


  0%|          | 0/167 [00:00<?, ?it/s]

TTA 1


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:51: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/167 [00:00<?, ?it/s]

TTA 2


100%|██████████| 167/167 [00:21<00:00,  7.75it/s]

finished


# see csv

In [18]:
with open('./data/result_3TTA.csv') as f4:
    lines4 = f4.readlines()
    print('csv_len: ',len(lines4))
    print(lines4)

csv_len:  5326
['Id,Category\n', '8820.png,jason wu\n', '348.png,alexander wang\n', '1804.png,chado ralph rucci\n', '4968.png,dries van noten\n', '8834.png,jil sander\n', '6831.png,missoni\n', '12907.png,carolina herrera\n', '3791.png,versace\n', '4798.png,givenchy\n', '7285.png,giorgio armani\n', '10862.png,yves saint laurent\n', '4954.png,yves saint laurent\n', '5492.png,missoni\n', '3785.png,chado ralph rucci\n', '8808.png,yves saint laurent\n', '12913.png,carolina herrera\n', '9525.png,derek lam\n', '11229.png,marni\n', '10137.png,alexander mcqueen\n', '5479.png,missoni\n', '6170.png,missoni\n', '8149.png,bottega veneta\n', '10123.png,chado ralph rucci\n', '14345.png,marc by marc jacobs\n', '5323.png,ralph lauren\n', '1145.png,armani prive\n', '9519.png,carolina herrera\n', '7534.png,marni\n', '7252.png,marc by marc jacobs\n', '1623.png,bottega veneta\n', '13364.png,yves saint laurent\n', '5445.png,missoni\n', '3034.png,jason wu\n', '5451.png,missoni\n', '11567.png,givenchy\n', '81